In [2]:
from tensorflow.keras.models import load_model
from utils import audio2spectrogram, load_audio_to_spectrogram
import time
import sounddevice as sd
from utils import audio2spectrogram, eminem_light
import numpy as np
from hue_functions import set_color_all, get_light_state, get_connected_lights
from user import bridge_url, hue_user

#### Load crnn model

In [3]:
model = load_model('best_model_3_sec.h5')

In [7]:
time_step = 6
starttime=time.time()

sr = 16000
seconds = 3
frames = int(seconds * sr)

# Get current hue and saturation value for each connected light
lights = get_connected_lights(bridge_url, hue_user)
previous_color = [[get_light_state(l)[key] for key in ['hue','sat']] for l in lights]
state = 'not eminem'

try:
    while True:
        print(time.ctime())
        recording = sd.rec(frames=frames, samplerate=sr, channels=1)
        sd.wait()
        recording = recording.reshape(-1)
        spec = audio2spectrogram(recording)
        prediction = np.random.randint(2, size=1)
        #prediction = np.argmax(model.predict(spec)[0])
        if not (state == 'not eminem' and prediction == 0):
            state, previous_color = eminem_light(bridge_url, hue_user, prediction, state, previous_color)
            time.sleep(time_step - ((time.time() - starttime) % time_step))
except KeyboardInterrupt:
    print('Stopped by user')

Fri Apr 10 18:01:06 2020
Fri Apr 10 18:01:10 2020
Fri Apr 10 18:01:18 2020
Fri Apr 10 18:01:24 2020
Stopped by user
